In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../src')

import math
import cadquery as cq
from ocp_vscode import show

from chapel2.dome_generator import (
    generate_dome_with_hubs,
    generate_chapel_dome,
)

from chapel2.geometry import (
    generate_honeycomb_dome,
    normalize, sub, add, scale, cross, norm, build_vertex_to_edges_map
)

from chapel2.hubs import (
    compute_hub_geometry,
    create_hub_by_style,
)

from chapel2.cuboid_struts import (
    create_shortened_cuboid_strut
)

from chapel2.miter_struts import (
    create_miter_cut_strut,
    compute_miter_cut_angles,
    analyze_miter_cuts_for_dome,
)


## Section 1: Simple 3-Strut Test Case

First, let's create a simple test case with just one vertex and 3 struts meeting at it.
This makes it easy to inspect the hub geometry in detail.


In [3]:
# Generate a small dome to get a single vertex with 3 struts
radius_cm = 50.0
frequency = 2
strut_width = 3.0  # cm
strut_depth = 3.0  # cm
hub_inset = strut_width * 0.5

vertices, faces, edges = generate_honeycomb_dome(radius_cm, frequency, 0.5)
vertex_to_edges = build_vertex_to_edges_map(edges)

print(f"Dome geometry: {len(vertices)} vertices, {len(edges)} edges")

# Find a vertex with exactly 3 struts (interior vertex)
test_vertex_idx = None
for v_idx, edge_list in vertex_to_edges.items():
    if len(edge_list) == 3:
        test_vertex_idx = v_idx
        break

print(f"Using vertex {test_vertex_idx} with 3 struts")
test_vertex = vertices[test_vertex_idx]


Dome geometry: 58 vertices, 82 edges
Using vertex 55 with 3 struts


In [4]:
# Create the 3 struts meeting at this vertex
dome_center = (0, 0, 0)

connected_edges = vertex_to_edges[test_vertex_idx]
struts_at_vertex = []

for edge_idx in connected_edges:
    v1_idx, v2_idx = edges[edge_idx]
    start = vertices[v1_idx]
    end = vertices[v2_idx]
    
    strut = create_shortened_cuboid_strut(
        start, end, strut_width, strut_depth,
        hub_inset, hub_inset, dome_center
    )
    struts_at_vertex.append(strut)

print(f"Created {len(struts_at_vertex)} struts")

# Show the struts without hub - notice the gap at the vertex
struts_compound = cq.Compound.makeCompound(struts_at_vertex)
print("\nStruts meeting at vertex (without hub) - notice the gap:")
show(struts_compound)


Created 3 struts

Struts meeting at vertex (without hub) - notice the gap:
Using port 3939
+


In [5]:
# Compute hub geometry for this vertex
hub_info = compute_hub_geometry(
    test_vertex, test_vertex_idx, vertices, edges, vertex_to_edges,
    strut_width, strut_depth, hub_inset, dome_center
)

print(f"Hub info:")
print(f"  Vertex position: ({test_vertex[0]:.2f}, {test_vertex[1]:.2f}, {test_vertex[2]:.2f})")
print(f"  Number of struts: {hub_info['num_struts']}")
print(f"  Hub inset: {hub_info['hub_inset']:.2f} cm")


Hub info:
  Vertex position: (44.33, 8.33, -13.91)
  Number of struts: 3
  Hub inset: 1.50 cm


## Tapered Prism Hub

The tapered prism hub has:
- **Inner face**: Flat triangle lying in the tangent plane (perpendicular to radial)
- **Outer face**: Another triangle, offset along the radial direction
- **Side faces**: Connect corresponding edges, angled to meet strut end faces


In [17]:
# Create tapered prism hub
tapered_prism_hub = create_hub_by_style(
    hub_info, strut_width, strut_depth, dome_center, 
    hub_style="tapered_prism"
)

if tapered_prism_hub is not None:
    print("Tapered Prism Hub created successfully!")
    print("\nHub only (tapered prism):")
    show(tapered_prism_hub)
else:
    print("Failed to create tapered prism hub")

# Show struts + tapered prism hub together
if tapered_prism_hub is not None:
    print("Struts + Tapered Prism Hub:")
    show(struts_compound, tapered_prism_hub)


Tapered Prism Hub created successfully!

Hub only (tapered prism):
c
Struts + Tapered Prism Hub:
cc


In [40]:
# Generate Chapel dome with tapered prism hubs
print("Generating Chapel dome with TAPERED PRISM hubs...")
struts_prism, hubs_prism, info_prism = generate_dome_with_hubs(
    radius_cm=8.0 * 30.48,
    frequency=3,
    strut_width=3.5 * 2.54,
    strut_depth=3.5 * 2.54,
    dome_style="honeycomb",
    hub_style="tapered_prism",
    strut_style="cuboid"
)

print(f"  Struts: {info_prism['num_struts_generated']}")
print(f"  Hubs: {info_prism['num_hubs_generated']}")

# Visualize tapered prism dome
print("Chapel Dome with Tapered Prism Hubs:")
show(struts_prism, hubs_prism)


Generating Chapel dome with TAPERED PRISM hubs...
  Struts: 240
  Hubs: 168
Chapel Dome with Tapered Prism Hubs:
++


## Manufacturability Analysis - Unique Parts Count

A critical question for manufacturing is: **How many unique parts do we need to make?**

We'll analyze:
1. **Unique strut types** - grouped by length and cut angles
2. **Unique hub types** - grouped by number of struts and angles between them


In [41]:
from collections import Counter

def analyze_unique_struts(vertices, edges, dome_center, 
                          length_tol_cm=0.1, angle_tol_deg=0.5):
    """
    Analyze struts and group by similar dimensions.
    
    For cuboid struts: group by length only (ends are perpendicular cuts)
    For miter-cut struts: group by length + cut angles at both ends
    """
    strut_signatures = []
    
    for v1_idx, v2_idx in edges:
        v1 = vertices[v1_idx]
        v2 = vertices[v2_idx]
        
        # Strut length
        length = norm(sub(v2, v1))
        length_rounded = round(length / length_tol_cm) * length_tol_cm
        
        # Cuboid struts - just length matters
        signature = (length_rounded,)
        
        strut_signatures.append(signature)
    
    # Count unique signatures
    signature_counts = Counter(strut_signatures)
    
    return {
        'total_struts': len(edges),
        'unique_types': len(signature_counts),
        'signature_counts': dict(signature_counts),
    }


def analyze_unique_hubs(vertices, edges, vertex_to_edges, dome_center, 
                        angle_tol_deg=1.0):
    """
    Analyze hubs and group by similar configurations.
    
    Hubs are characterized by:
    - Number of struts
    - Angles between adjacent struts (sorted)
    """
    hub_signatures = []
    
    for v_idx, edge_indices in vertex_to_edges.items():
        vertex = vertices[v_idx]
        num_struts = len(edge_indices)
        
        if num_struts < 2:
            continue
        
        # Get strut directions
        directions = []
        for edge_idx in edge_indices:
            v1_idx, v2_idx = edges[edge_idx]
            other_idx = v2_idx if v1_idx == v_idx else v1_idx
            other = vertices[other_idx]
            direction = normalize(sub(other, vertex))
            directions.append(direction)
        
        # Compute angles between all pairs of struts
        angles = []
        for i in range(len(directions)):
            for j in range(i+1, len(directions)):
                d1, d2 = directions[i], directions[j]
                cos_angle = d1[0]*d2[0] + d1[1]*d2[1] + d1[2]*d2[2]
                angle_deg = math.degrees(math.acos(max(-1, min(1, cos_angle))))
                angle_rounded = round(angle_deg / angle_tol_deg) * angle_tol_deg
                angles.append(angle_rounded)
        
        # Sort angles to create a canonical signature
        angles_sorted = tuple(sorted(angles))
        
        signature = (num_struts, angles_sorted)
        hub_signatures.append(signature)
    
    signature_counts = Counter(hub_signatures)
    
    return {
        'total_hubs': len(hub_signatures),
        'unique_types': len(signature_counts),
        'signature_counts': dict(signature_counts),
    }


In [42]:
# Analyze Chapel dome parts
print("="*70)
print("MANUFACTURABILITY ANALYSIS - CHAPEL DOME (8ft, 3V)")
print("="*70)

# Get dome geometry
radius_cm_analysis = 8.0 * 30.48
vertices_analysis, faces_analysis, edges_analysis = generate_honeycomb_dome(radius_cm_analysis, 3, 0.5)
vertex_to_edges_analysis = build_vertex_to_edges_map(edges_analysis)
dome_center_analysis = (0, 0, 0)

print(f"\nDome Statistics:")
print(f"  Total vertices: {len(vertices_analysis)}")
print(f"  Total edges (struts): {len(edges_analysis)}")
print(f"  Total faces (windows): {len(faces_analysis)}")


MANUFACTURABILITY ANALYSIS - CHAPEL DOME (8ft, 3V)

Dome Statistics:
  Total vertices: 168
  Total edges (struts): 240
  Total faces (windows): 73


In [43]:
# Analyze struts for cuboid style (Option 2: Tapered Prism)
print("\n" + "-"*70)
print("TAPERED PRISM HUBS + CUBOID STRUTS")
print("-"*70)

cuboid_analysis = analyze_unique_struts(vertices_analysis, edges_analysis, dome_center_analysis)
print(f"\nStrut Analysis:")
print(f"  Total struts: {cuboid_analysis['total_struts']}")
print(f"  Unique strut types: {cuboid_analysis['unique_types']}")
print(f"\n  Cut list (length -> count):")
for sig, count in sorted(cuboid_analysis['signature_counts'].items()):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    print(f"    {length_cm:.1f} cm ({length_in:.1f} in): {count} pieces")



----------------------------------------------------------------------
TAPERED PRISM HUBS + CUBOID STRUTS
----------------------------------------------------------------------

Strut Analysis:
  Total struts: 240
  Unique strut types: 6

  Cut list (length -> count):
    37.3 cm (14.7 in): 32 pieces
    41.4 cm (16.3 in): 60 pieces
    42.2 cm (16.6 in): 28 pieces
    42.6 cm (16.8 in): 60 pieces
    42.7 cm (16.8 in): 28 pieces
    44.4 cm (17.5 in): 32 pieces


In [44]:
# Analyze hubs for tapered prism
hub_analysis = analyze_unique_hubs(vertices_analysis, edges_analysis, vertex_to_edges_analysis, dome_center_analysis)
print(f"\nHub Analysis:")
print(f"  Total hubs: {hub_analysis['total_hubs']}")
print(f"  Unique hub types: {hub_analysis['unique_types']}")
print(f"\n  Hub types (struts, angles) -> count:")
for sig, count in sorted(hub_analysis['signature_counts'].items()):
    num_struts = sig[0]
    angles = sig[1]
    angles_str = ", ".join(f"{a:.0f} deg" for a in angles)
    print(f"    {num_struts}-strut hub [{angles_str}]: {count} pieces")



Hub Analysis:
  Total hubs: 168
  Unique hub types: 10

  Hub types (struts, angles) -> count:
    2-strut hub [115 deg]: 4 pieces
    2-strut hub [117 deg]: 4 pieces
    2-strut hub [119 deg]: 4 pieces
    2-strut hub [120 deg]: 4 pieces
    2-strut hub [125 deg]: 8 pieces
    3-strut hub [108 deg, 125 deg, 125 deg]: 24 pieces
    3-strut hub [115 deg, 115 deg, 127 deg]: 28 pieces
    3-strut hub [117 deg, 120 deg, 121 deg]: 56 pieces
    3-strut hub [118 deg, 120 deg, 120 deg]: 28 pieces
    3-strut hub [119 deg, 119 deg, 119 deg]: 8 pieces


In [23]:
# Manufacturing notes
print("\n" + "="*70)
print("MANUFACTURING NOTES")
print("="*70)
print("""
TAPERED PRISM (Option 2):
  + Simpler strut cuts (perpendicular only)
  + Clean flat inner surface for aesthetics
  - Many unique hub shapes (one for each unique angle configuration)
  - Hub shapes are irregular polyhedra (harder to manufacture)
  - Best for: 3D printing or CNC machining hubs
""")



MANUFACTURING NOTES

TAPERED PRISM (Option 2):
  + Simpler strut cuts (perpendicular only)
  + Clean flat inner surface for aesthetics
  - Many unique hub shapes (one for each unique angle configuration)
  - Hub shapes are irregular polyhedra (harder to manufacture)
  - Best for: 3D printing or CNC machining hubs



## Detailed Cut Lists for Manufacturing

Generate detailed cut lists that can be used for actual manufacturing.


In [24]:
# Generate detailed cut list for cuboid struts
print("="*70)
print("DETAILED CUT LIST - CUBOID STRUTS (Option 2: Tapered Prism)")
print("="*70)
print(f"\nStrut cross-section: 4 in x 4 in (10.16 cm x 10.16 cm)")
print(f"Cut type: Perpendicular (90 deg) at both ends")
print(f"\n{'Type':<6} {'Length (cm)':<12} {'Length (in)':<12} {'Count':<8} {'Notes'}")
print("-"*70)

for i, (sig, count) in enumerate(sorted(cuboid_analysis['signature_counts'].items()), 1):
    length_cm = sig[0]
    length_in = length_cm / 2.54
    print(f"A{i:<5} {length_cm:<12.1f} {length_in:<12.1f} {count:<8} Square cut both ends")

total_length_cm = sum(sig[0] * count for sig, count in cuboid_analysis['signature_counts'].items())
total_length_ft = total_length_cm / 30.48
print("-"*70)
print(f"TOTAL: {cuboid_analysis['total_struts']} struts, {total_length_ft:.0f} linear feet of material")


DETAILED CUT LIST - CUBOID STRUTS (Option 2: Tapered Prism)

Strut cross-section: 4 in x 4 in (10.16 cm x 10.16 cm)
Cut type: Perpendicular (90 deg) at both ends

Type   Length (cm)  Length (in)  Count    Notes
----------------------------------------------------------------------
A1     37.3         14.7         36       Square cut both ends
A2     41.4         16.3         72       Square cut both ends
A3     42.2         16.6         40       Square cut both ends
A4     42.6         16.8         68       Square cut both ends
A5     42.7         16.8         34       Square cut both ends
A6     44.4         17.5         36       Square cut both ends
----------------------------------------------------------------------
TOTAL: 286 struts, 392 linear feet of material


In [25]:
# Generate hub list for tapered prism hubs
print("\n" + "="*70)
print("DETAILED HUB LIST - TAPERED PRISM HUBS (Option 2)")
print("="*70)
print(f"\nHub material: Custom 3D printed or CNC machined")
print(f"\n{'Type':<6} {'Struts':<8} {'Angles':<40} {'Count':<8}")
print("-"*70)

for i, (sig, count) in enumerate(sorted(hub_analysis['signature_counts'].items()), 1):
    num_struts = sig[0]
    angles = sig[1]
    angles_str = ", ".join(f"{a:.0f} deg" for a in angles)
    print(f"H{i:<5} {num_struts:<8} {angles_str:<40} {count:<8}")

print("-"*70)
print(f"TOTAL: {hub_analysis['total_hubs']} hubs in {hub_analysis['unique_types']} unique types")



DETAILED HUB LIST - TAPERED PRISM HUBS (Option 2)

Hub material: Custom 3D printed or CNC machined

Type   Struts   Angles                                   Count   
----------------------------------------------------------------------
H1     2        108 deg                                  4       
H2     2        115 deg                                  4       
H3     2        117 deg                                  4       
H4     2        118 deg                                  2       
H5     2        120 deg                                  4       
H6     2        121 deg                                  4       
H7     3        108 deg, 125 deg, 125 deg                36      
H8     3        115 deg, 115 deg, 127 deg                32      
H9     3        117 deg, 120 deg, 121 deg                64      
H10    3        118 deg, 120 deg, 120 deg                32      
H11    3        119 deg, 119 deg, 119 deg                12      
------------------------------------